# Teksto klasifikavimo užduotis

Kaip jau minėjome, mes sutelksime dėmesį į paprastą teksto klasifikavimo užduotį, pagrįstą **AG_NEWS** duomenų rinkiniu, kurio tikslas yra klasifikuoti naujienų antraštes į vieną iš 4 kategorijų: Pasaulis, Sportas, Verslas ir Mokslas/Technologijos.

## Duomenų rinkinys

Šis duomenų rinkinys yra integruotas į [`torchtext`](https://github.com/pytorch/text) modulį, todėl galime lengvai jį pasiekti.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Čia `train_dataset` ir `test_dataset` yra kolekcijos, kurios grąžina poras: etiketę (klasės numerį) ir tekstą atitinkamai, pavyzdžiui:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

Taigi, atspausdinkime pirmąsias 10 naujų antraščių iš mūsų duomenų rinkinio:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

Kadangi duomenų rinkiniai yra iteratoriai, jei norime naudoti duomenis kelis kartus, turime juos konvertuoti į sąrašą:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Tokenizacija

Dabar turime paversti tekstą į **skaičius**, kuriuos galima atvaizduoti kaip tensorius. Jei norime žodžių lygmens reprezentacijos, turime atlikti du dalykus:
* naudoti **tokenizatorių**, kad tekstas būtų padalintas į **tokenus**
* sukurti tų tokenų **žodyną**.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

Naudodami žodyną, galime lengvai užkoduoti savo suskaidytą eilutę į skaičių rinkinį:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Žodžių maišo teksto reprezentacija

Kadangi žodžiai perteikia prasmę, kartais teksto prasmę galime suprasti tiesiog pažvelgę į atskirus žodžius, nepaisant jų tvarkos sakinyje. Pavyzdžiui, klasifikuojant naujienas, tokie žodžiai kaip *oras*, *sniegas* greičiausiai nurodys *orų prognozę*, o žodžiai kaip *akcijos*, *doleris* būtų susiję su *finansinėmis naujienomis*.

**Žodžių maišo** (BoW) vektorinė reprezentacija yra dažniausiai naudojama tradicinė vektorinė reprezentacija. Kiekvienas žodis yra susietas su vektoriaus indeksu, o vektoriaus elementas nurodo, kiek kartų žodis pasirodo tam tikrame dokumente.

![Vaizdas, rodantis, kaip žodžių maišo vektorinė reprezentacija saugoma atmintyje.](../../../../../lessons/5-NLP/13-TextRep/images/bag-of-words-example.png) 

> **Note**: Taip pat galite galvoti apie BoW kaip apie visų vieno žodžio koduotų vektorių sumą tekste.

Žemiau pateiktas pavyzdys, kaip sukurti žodžių maišo reprezentaciją naudojant Scikit Learn python biblioteką:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Norėdami apskaičiuoti žodžių maišo vektorių iš mūsų AG_NEWS duomenų rinkinio vektorinės reprezentacijos, galime naudoti šią funkciją:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Pastaba:** Čia naudojame globalųjį kintamąjį `vocab_size`, kad nurodytume numatytąjį žodyno dydį. Kadangi dažnai žodyno dydis yra gana didelis, galime apriboti žodyno dydį iki dažniausiai vartojamų žodžių. Pabandykite sumažinti `vocab_size` reikšmę ir paleisti žemiau pateiktą kodą, kad pamatytumėte, kaip tai veikia tikslumą. Turėtumėte tikėtis tam tikro tikslumo sumažėjimo, tačiau ne drastiško, mainais už didesnį našumą.


## Mokome BoW klasifikatorių

Dabar, kai išmokome sukurti Bag-of-Words (BoW) reprezentaciją mūsų tekstui, apmokykime klasifikatorių, naudodami šią reprezentaciją. Pirmiausia turime konvertuoti savo duomenų rinkinį mokymui taip, kad visos pozicinės vektorinės reprezentacijos būtų paverstos į Bag-of-Words reprezentaciją. Tai galima padaryti perduodant funkciją `bowify` kaip `collate_fn` parametrą standartiniam torch `DataLoader`:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Dabar apibrėžkime paprastą klasifikatoriaus neuroninį tinklą, kuris turi vieną linijinį sluoksnį. Įvesties vektoriaus dydis yra lygus `vocab_size`, o išvesties dydis atitinka klasių skaičių (4). Kadangi sprendžiame klasifikavimo užduotį, galutinė aktyvavimo funkcija yra `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Dabar apibrėšime standartinį PyTorch mokymo ciklą. Kadangi mūsų duomenų rinkinys yra gana didelis, mokymo tikslais treniruosime tik vieną epochą, o kartais net mažiau nei vieną epochą (nustatant `epoch_size` parametrą galima apriboti mokymą). Taip pat pranešime apie sukauptą mokymo tikslumą mokymo metu; pranešimo dažnis nustatomas naudojant `report_freq` parametrą.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## BiGramai, TriGramai ir N-Gramai

Viena iš maišo žodžių metodo apribojimų yra ta, kad kai kurie žodžiai sudaro daugiakalbius posakius. Pavyzdžiui, žodis „hot dog“ turi visiškai kitokią reikšmę nei žodžiai „hot“ ir „dog“ kitame kontekste. Jei žodžius „hot“ ir „dog“ visada atvaizduosime tais pačiais vektoriais, tai gali suklaidinti mūsų modelį.

Norint tai išspręsti, dokumentų klasifikavimo metodai dažnai naudoja **N-gramų reprezentacijas**, kur kiekvieno žodžio, dviejų žodžių ar trijų žodžių dažnis yra naudinga savybė mokant klasifikatorius. Pavyzdžiui, bigramų reprezentacijoje mes į žodyną pridėsime visas žodžių poras, be originalių žodžių.

Žemiau pateiktas pavyzdys, kaip sukurti bigramų maišo žodžių reprezentaciją naudojant Scikit Learn:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Pagrindinis N-gram metodo trūkumas yra tas, kad žodyno dydis pradeda augti itin greitai. Praktikoje reikia derinti N-gram reprezentaciją su tam tikromis dimensijų mažinimo technikomis, tokiomis kaip *embedding'ai*, apie kurias kalbėsime kitame skyriuje.

Norint naudoti N-gram reprezentaciją mūsų **AG News** duomenų rinkinyje, reikia sukurti specialų ngram žodyną:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


Tą patį kodą galėtume naudoti klasifikatoriui mokyti, tačiau tai būtų labai neefektyvu atminties atžvilgiu. Kitame skyriuje mokysime bigramų klasifikatorių naudodami įterpimus.

> **Pastaba:** Galite palikti tik tuos ngramus, kurie tekste pasirodo daugiau nei nurodytą kartų skaičių. Tai užtikrins, kad retai pasitaikantys bigramai bus praleisti, ir žymiai sumažins dimensionalumą. Norėdami tai padaryti, nustatykite `min_freq` parametrą į didesnę reikšmę ir stebėkite, kaip keičiasi žodyno ilgis.


## Termų dažnio ir atvirkštinio dokumentų dažnio metodas (TF-IDF)

BoW (maišo žodžių) reprezentacijoje žodžių pasikartojimai yra vertinami vienodai, nepriklausomai nuo paties žodžio. Tačiau akivaizdu, kad dažnai pasitaikantys žodžiai, tokie kaip *a*, *in* ir pan., yra daug mažiau svarbūs klasifikacijai nei specializuoti terminai. Iš tiesų, daugelyje NLP užduočių kai kurie žodžiai yra reikšmingesni nei kiti.

**TF-IDF** reiškia **termų dažnio–atvirkštinio dokumentų dažnio metodą**. Tai yra maišo žodžių variacija, kur vietoj dvejetainės 0/1 reikšmės, nurodančios žodžio pasirodymą dokumente, naudojama slankiojo kablelio reikšmė, susijusi su žodžio pasikartojimo dažniu korpuse.

Formaliau, žodžio $i$ svoris dokumente $j$ apibrėžiamas taip:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
kur:
* $tf_{ij}$ yra žodžio $i$ pasikartojimų skaičius dokumente $j$, t. y. BoW reikšmė, kurią jau aptarėme
* $N$ yra dokumentų skaičius kolekcijoje
* $df_i$ yra dokumentų, kuriuose yra žodis $i$, skaičius visoje kolekcijoje

TF-IDF reikšmė $w_{ij}$ didėja proporcingai žodžio pasikartojimų skaičiui dokumente ir yra koreguojama pagal dokumentų skaičių korpuse, kuriuose yra tas žodis. Tai padeda atsižvelgti į tai, kad kai kurie žodžiai pasitaiko dažniau nei kiti. Pavyzdžiui, jei žodis pasirodo *kiekviename* kolekcijos dokumente, $df_i=N$, ir $w_{ij}=0$, todėl tokie terminai būtų visiškai ignoruojami.

TF-IDF tekstų vektorizaciją galite lengvai sukurti naudodami Scikit Learn:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## Išvada

Nors TF-IDF reprezentacijos suteikia skirtingiems žodžiams dažnio svorį, jos nesugeba perteikti prasmės ar tvarkos. Kaip garsus lingvistas J. R. Firth 1935 m. pasakė: „Visapusiška žodžio prasmė visada yra kontekstinė, ir joks prasmės tyrimas, atskirtas nuo konteksto, negali būti laikomas rimtu.“ Vėliau kurse sužinosime, kaip iš teksto išgauti kontekstinę informaciją naudojant kalbos modeliavimą.



---

**Atsakomybės apribojimas**:  
Šis dokumentas buvo išverstas naudojant AI vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, prašome atkreipti dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Kritinei informacijai rekomenduojama naudoti profesionalų žmogaus vertimą. Mes neprisiimame atsakomybės už nesusipratimus ar klaidingus interpretavimus, atsiradusius dėl šio vertimo naudojimo.
